In [1]:
from scipy import stats
from numpy import linspace, exp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from autumn.settings.constants import COVID_BASE_DATETIME
import datetime
from summer.utils import ref_times_to_dti
from autumn.core.inputs.mobility.queries import get_mobility_data
from autumn.models.sm_sir.mixing_matrix.macrodistancing import weight_mobility_data
from autumn.core.inputs.database import get_input_db

from autumn.core import inputs
from notebooks.user.rragonnet.project_specific.School_Closure.plotting_constants import (
    SCHOOL_PROJECT_NOTEBOOK_PATH, 
    FIGURE_WIDTH,
    RESOLUTION,
    INCLUDED_COUNTRIES,
    set_up_style
)
from autumn.models.sm_covid import base_params
from autumn.core.inputs.social_mixing.build_synthetic_matrices import get_matrices_from_conmat
import os

set_up_style()
country_input_fig_path = os.path.join(SCHOOL_PROJECT_NOTEBOOK_PATH, "country_input_figs")

In [3]:
def save_figure(name, dir_path=country_input_fig_path, formats=["png", "pdf"]):
    for format in formats:
        path = os.path.join(dir_path, f"{name}.{format}")
        plt.savefig(path, facecolor="white")

## Google mobility

In [7]:
dir_path = os.path.join(country_input_fig_path, "mobility")
os.makedirs(dir_path, exist_ok=True)

input_db = get_input_db()

loc_names = {
    "work": "workplace",
    "other_locations": "other locations"
}
plot_left_date = datetime.date(2020, 1, 1)
# plot_right_date = times[-1]  # Not sure why this is necessary

# iso3_list = ["FRA"]
iso3_list = list(INCLUDED_COUNTRIES.keys())

for iso3 in iso3_list:

    mob_df, int_times = get_mobility_data(iso3, None, COVID_BASE_DATETIME)
    google_mob_df = weight_mobility_data(mob_df, base_params["mobility"]["google_mobility_locations"])

    times = ref_times_to_dti(COVID_BASE_DATETIME, int_times)
    fig, ax = plt.subplots(1, 1, figsize=(0.8 * FIGURE_WIDTH, 0.6 * FIGURE_WIDTH), dpi=RESOLUTION)

    for location in list(base_params["mobility"]["google_mobility_locations"].keys()):
        ax.plot(times, google_mob_df[location].rolling(7).mean(), label=loc_names[location])
    ax.tick_params(axis="x", labelrotation=45)
    # ax.set_xlim(left=plot_left_date, right=plot_right_date)

    # ax.set_ylim((0., y_upper))
    ax.set_ylabel("mobility level")

    ax.legend(loc="upper left")
    ax.set_title("Google mobility data")

    plt.tight_layout(w_pad=1.5, h_pad=3.5)
    save_figure(f"mobility_{iso3}", dir_path)
    plt.close()

## Mixing matrix

In [10]:
dir_path = os.path.join(country_input_fig_path, "mixing")

age_groups = base_params['age_groups']
for iso3, country in INCLUDED_COUNTRIES.items():
    mixing_matrices = get_matrices_from_conmat(iso3, [int(age) for age in age_groups])

    fig = plt.figure(figsize=(FIGURE_WIDTH, FIGURE_WIDTH), dpi=RESOLUTION)
    positions = [1, 2, 3, 5, 6]    
    for i_loc, location in zip(positions, mixing_matrices.keys()):
        ax = fig.add_subplot(2, 3, i_loc)
        ax.imshow(
            np.flipud(np.transpose(mixing_matrices[location])), 
            cmap=cm.hot, 
            vmin=0,
            vmax=mixing_matrices[location].max(), 
            origin="lower"
        )
        ax.set_title(location.replace("_", " "))
        ax.set_xticks([])
        ax.set_yticks([])

    plt.tight_layout(w_pad=1.5, h_pad=3.5)
    save_figure(f"mixing_{iso3}", dir_path)
    plt.close()
    break

## Vaccine coverage